<a href="https://colab.research.google.com/github/greeanday/newrepo/blob/master/subject1_Q6_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Q6**
From each window, extract energy, Shannon entropy, maximum (amplitude and
location), minimum (amplitude and location). Store these as a separate matrix
where rows correspond to windows and columns correspond to features (there
will be 6 features).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dataFrame=pd.read_csv("subject101.dat",sep=" ",header=None)

dataFrame.shape[1]

dataFrame.columns=["timestamp","activityID","heart_rate"]+[f'col_{i}' for i in range(4,55)]

dataFrame["col_40"]=pd.to_numeric(dataFrame["col_40"], errors="coerce")
dataFrame["col_41"]=pd.to_numeric(dataFrame["col_41"], errors="coerce")
dataFrame["col_42"]=pd.to_numeric(dataFrame["col_42"], errors="coerce")

walking=dataFrame[dataFrame["activityID"]==4]
xyz = walking[["timestamp","col_40", "col_41", "col_42"]].dropna()

magnitude = np.sqrt(xyz["col_40"]**2 + xyz["col_41"]**2 + xyz["col_42"]**2)


rows = []
sampling_rate = 100  # Hz
segment_length = 4 * sampling_rate  # 4 saniye = 400 örnek

for i in range(0, len(magnitude), segment_length):
    segment = magnitude.iloc[i:i+segment_length].values  # .iloc kullandık
    if len(segment) < segment_length:
        segment = np.pad(segment, (0, segment_length - len(segment)), constant_values=np.nan)
    rows.append(segment)

matrix_4sec_segments = np.array(rows)



In [ ]:
import numpy as np
from scipy.stats import entropy

# Özellikleri saklayacağımız liste
features = []

# Her pencere için özellikleri hesapla
for segment in matrix_4sec_segments:
    # NaN'leri filtrele
    segment = segment[~np.isnan(segment)]

    # Enerji: karelerin toplamı
    energy = np.sum(segment**2)

    # Shannon Entropy
    # Frekansları say (benzersiz değerleri bularak)
    values, counts = np.unique(segment, return_counts=True)
    probabilities = counts / counts.sum()
    shannon_entropy = entropy(probabilities, base=2)

    # Maksimum ve konumu
    max_val = np.max(segment)
    max_idx = np.argmax(segment)

    # Minimum ve konumu
    min_val = np.min(segment)
    min_idx = np.argmin(segment)

    # Özellikleri birleştir ve ekle
    features.append([energy, shannon_entropy, max_val, max_idx, min_val, min_idx])

# Özellik matrisi olarak numpy array'e çevir
feature_matrix = np.array(features)

# Dilersen bir DataFrame'e de çevirebilirsin:
import pandas as pd
columns = ['Energy', 'ShannonEntropy', 'MaxValue', 'MaxIndex', 'MinValue', 'MinIndex']
feature_df = pd.DataFrame(feature_matrix, columns=columns)

In [ ]:
print(feature_df)

           Energy  ShannonEntropy    MaxValue  MaxIndex  MinValue  MinIndex
0    38825.397754        8.643856    9.973791     189.0  9.677275     375.0
1    70680.515963        8.643856   91.371940     338.0  3.668466     301.0
2   136371.515093        8.643856   88.221441      57.0  3.337948     184.0
3   161342.521612        8.643856  109.446975     128.0  3.757926      94.0
4   156749.685519        8.643856  137.828835     192.0  3.798703     160.0
5   165124.626119        8.643856  111.950211     376.0  3.710919      37.0
6   141759.061064        8.643856   87.789681     321.0  3.720366      56.0
7   163157.154537        8.643856   97.326597     378.0  3.604315       4.0
8   160387.086520        8.643856  145.152143     206.0  3.819535      58.0
9   176882.785109        8.643856  123.000840      36.0  3.879249       3.0
10  134680.495691        8.643856  108.419613     334.0  4.001636     178.0
11  167311.872256        8.643856  141.715731      56.0  3.605557     372.0
12  140810.4

SHANNON ENTROPİSİNİ HESAPLARKEN .unique ile her bir valueden kaç tane olduğunu sayıp sonra toplam count sayısına bölüp probabilityleri entropi denklemine dahil ettik evet. ama float sayıs olduğu için ve bu yüzden ufak farklarda bile farklı sayıldığı için entropi olduğundan fazla olarak yüksek görünmüş olabilir. Bu. yüzeen histogram ile bin'lere ayırarak ilerliyeceğiz. Binleri de herbir segment (time window)için ayrı ayrı Freedman-Diaconis Kuralı ile hesaplayıp o şekilde oluşturacağız.

"probabilities = counts / counts.sum() = [1/400, 1/400, ..., 1/400]
log 2(400)≈8.64 bit
Entropi yaklaşık 8.64 bit olur.
Bu, Shannon entropisinin o segmentteki maksimum belirsizlik durumudur." yani aslında her bir değer farklıymış ki entropi hepsinde 8.64 çıktı yukarda kontrol ettiğimde.

In [ ]:
import numpy as np
from scipy.stats import entropy, iqr

# Özellikleri saklayacağımız liste
features = []

# Her pencere için özellikleri hesapla
for segment in matrix_4sec_segments:
    # NaN'leri filtrele
    segment = segment[~np.isnan(segment)]

    # Enerji: karelerin toplamı
    energy = np.sum(segment**2)

    # Shannon Entropy
    # Frekansları say (Freedman-Diaconis bin sayısı bularak)
    segment_iqr=iqr(segment)
    bin_width=segment_iqr/(len(segment)**(1/3))

    #bin sayısı hesqaplayalım
    bin_count=int(np.ceil((segment.max()-segment.min()) / bin_width))

    #histogram hesaplayalım
    counts, _ =np.histogram(segment,bins=bin_count, density=True)

    #olasılıklar ve entropi hesaplayalım

    probabilities=counts/np.sum(counts)
    shannon_entropy=entropy(probabilities, base=2)



    # Maksimum ve konumu
    max_val = np.max(segment)
    max_idx = np.argmax(segment)

    # Minimum ve konumu
    min_val = np.min(segment)
    min_idx = np.argmin(segment)

    # Özellikleri birleştir ve ekle
    features.append([energy, shannon_entropy, max_val, max_idx, min_val, min_idx])

# Özellik matrisi olarak numpy array'e çevir
feature_matrix = np.array(features)

# Dilersen bir DataFrame'e de çevirebilirsin:
import pandas as pd
columns = ['Energy', 'ShannonEntropy', 'MaxValue', 'MaxIndex', 'MinValue', 'MinIndex']
feature_df = pd.DataFrame(feature_matrix, columns=columns)

In [ ]:
print(counts)
print(bin_count)
print(probabilities)
print(feature_df)

[ 0.83599295  0.          2.50797885  1.6719859   5.85195065  5.85195065
 10.86790835  5.85195065  6.6879436  15.88386605 12.53989425 20.89982375
 10.0319154  11.7039013  13.3758872   8.3599295   5.0159577   1.6719859
  4.17996475  7.52393655  1.6719859   1.6719859 ]
22
[0.00540541 0.         0.01621622 0.01081081 0.03783784 0.03783784
 0.07027027 0.03783784 0.04324324 0.1027027  0.08108108 0.13513514
 0.06486486 0.07567568 0.08648649 0.05405405 0.03243243 0.01081081
 0.02702703 0.04864865 0.01081081 0.01081081]
           Energy  ShannonEntropy    MaxValue  MaxIndex  MinValue  MinIndex
0    38825.397754        4.576796    9.973791     189.0  9.677275     375.0
1    70680.515963        4.536572   91.371940     338.0  3.668466     301.0
2   136371.515093        4.122369   88.221441      57.0  3.337948     184.0
3   161342.521612        3.989211  109.446975     128.0  3.757926      94.0
4   156749.685519        4.074197  137.828835     192.0  3.798703     160.0
5   165124.626119        4

# count sayımı olasılık yoğunluk fonksiyonu (PDF) üzerinden mi yapılmalı yoksa normal (density=False diyip) tam sayılı count üzerinden mi olmalı??

not: (density=True) diyince PDF, (density=False) diyince tam sayılı normal o aralıkta kaç veri olduğunu gösteren şekilde count veriyor.

# Şimdi de density=False diyeceğim ve bin_width aralığındaki veri miktarını bularak count sayısı tam sayı olunca entropy değişimi var mı vs diye bakacağıım. ***Ve Beren hocaya hangi  şekilde ilerlemem gerektiğini sormayı unutma***

In [ ]:
import numpy as np
from scipy.stats import entropy, iqr

# Özellikleri saklayacağımız liste
features = []

# Her pencere için özellikleri hesapla
for segment in matrix_4sec_segments:
    # NaN'leri filtrele
    segment = segment[~np.isnan(segment)]

    # Enerji: karelerin toplamı
    energy = np.sum(segment**2)

    # Shannon Entropy
    # Frekansları say (Freedman-Diaconis bin sayısı bularak)
    segment_iqr=iqr(segment)
    bin_width=segment_iqr/(len(segment)**(1/3))

    #bin sayısı hesqaplayalım
    bin_count=int(np.ceil((segment.max()-segment.min()) / bin_width))

    #histogram hesaplayalım
    counts, _ =np.histogram(segment,bins=bin_count, density=False)

    #olasılıklar ve entropi hesaplayalım

    probabilities=counts/np.sum(counts)
    shannon_entropy=entropy(probabilities, base=2)



    # Maksimum ve konumu
    max_val = np.max(segment)
    max_idx = np.argmax(segment)

    # Minimum ve konumu
    min_val = np.min(segment)
    min_idx = np.argmin(segment)

    # Özellikleri birleştir ve ekle
    features.append([energy, shannon_entropy, max_val, max_idx, min_val, min_idx])

# Özellik matrisi olarak numpy array'e çevir
feature_matrix = np.array(features)

# Dilersen bir DataFrame'e de çevirebilirsin:
import pandas as pd
columns = ['Energy', 'ShannonEntropy', 'MaxValue', 'MaxIndex', 'MinValue', 'MinIndex']
feature_df_density_false = pd.DataFrame(feature_matrix, columns=columns)

In [ ]:
print(counts)
print(bin_count)
print(probabilities)
print(feature_df_density_false)

[ 1  0  3  2  7  7 13  7  8 19 15 25 12 14 16 10  6  2  5  9  2  2]
22
[0.00540541 0.         0.01621622 0.01081081 0.03783784 0.03783784
 0.07027027 0.03783784 0.04324324 0.1027027  0.08108108 0.13513514
 0.06486486 0.07567568 0.08648649 0.05405405 0.03243243 0.01081081
 0.02702703 0.04864865 0.01081081 0.01081081]
           Energy  ShannonEntropy    MaxValue  MaxIndex  MinValue  MinIndex
0    38825.397754        4.576796    9.973791     189.0  9.677275     375.0
1    70680.515963        4.536572   91.371940     338.0  3.668466     301.0
2   136371.515093        4.122369   88.221441      57.0  3.337948     184.0
3   161342.521612        3.989211  109.446975     128.0  3.757926      94.0
4   156749.685519        4.074197  137.828835     192.0  3.798703     160.0
5   165124.626119        4.055019  111.950211     376.0  3.710919      37.0
6   141759.061064        4.029447   87.789681     321.0  3.720366      56.0
7   163157.154537        4.036215   97.326597     378.0  3.604315       4.

# **Q7**
Write this matrix into a .csv file. Make sure that you have a header for the feature
names.

In [ ]:
feature_df.to_csv('feature_matrix_DensityTrue.csv', index=True)

In [ ]:
feature_df_density_false.to_csv("feature_matrix_DensityFalse.csv", index=True)

# **Q8**
For the IMU chest acceleration Z, filter the signal between 0.5 – 40 Hz. Plot the
spectrum (frequency domain representation) and try to locate the dominant
frequencies for the resultant signal.